In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Session 5: NN with one hidden layer

<p style="color:blue;">
By Pramod Sharma : pramod.sharma@prasami.com
<p>

In [ ]:
# Lets import some libraries
import os

import numpy as np

import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn import datasets, linear_model

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.metrics import plot_confusion_matrix

%matplotlib inline

In [ ]:
# Some basic parameters

inpDir = '../input' # location where input data is stored
outDir = '../output' # location to store outputs
RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production
EPOCHS = 20000 # number of cycles to run

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (12,9),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

plt.rcParams.update(params) # update rcParams

## Generate Data Set
<p style="font-family: Arial; font-size:1.1em;color:blue;">
Use Sklearn's dataset generator <a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html">make_moon</a>.
</p>

In [ ]:
X, y = datasets.make_moons(n_samples=1280, shuffle=True, noise=0.2, random_state=RANDOM_STATE)

<p style="font-family: Arial; font-size:1.1em;color:brown;">
<strong>Note</strong>: All two dimensional matrix are represented by Caps and all arrays (vectors) are represented by small case.
</p>

In [ ]:
# Lets Plot the data
plt.scatter(X[:,0], X[:,1], s=30, c=y, cmap=plt.cm.Spectral)

plt.grid()

In [ ]:
def fn_plot_decision_boundary(pred_func, X, y):
    '''
        Attrib:
           pred_func : function based on predict method of the classifier
           X : feature matrix
           y : targets
       Return:
           None
    '''
    
    # Set min and max values and give it some padding
    xMin, xMax = X[:, 0].min() - .05, X[:, 0].max() + .05
    yMin, yMax = X[:, 1].min() - .05, X[:, 1].max() + .05
    
    # grid size for mesh grid
    h = 0.01
    
    # Generate a grid of points with distance 'h' between them
    xx, yy = np.meshgrid(np.arange(xMin, xMax, h), np.arange(yMin, yMax, h))
    
    # Predict the function value for the whole grid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    
    # Make its shape same as that of xx 
    Z = Z.reshape(xx.shape)
    
    # Now we have Z value corresponding to each of the combination of xx and yy
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)
    
    # plot the points as well
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, edgecolors='black')

In [ ]:
#  Split the data in training and test sets to measure performance of the model.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE )

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Neural Network
<p style="font-family: Arial; font-size:1.2em;color:black;"> 
    Let's start with simple network. Our data has <strong>two</strong> features. Hence size of input layer will also be two. The output is binary, we can code it as single column as well as double column output. The hidden layer could be of <strong>any size</strong>. One need to execute a handful of iterations to arrive at right size of hidden layer. For purpose of this demo, size of hidden layer is taken as shown below.
</p>
<img src='../Presentations/images/S5/nn_S5_f1.jpg' style='width: 100'/>

## Loss Function
<p style="font-family: Arial; font-size:1.2em;color:black;">
We need to minimise the error by adjusting ($W_1, b_1$). We call the function that measures our error the *loss function*. A common choice with the softmax output is the cross-entropy loss. The loss for our prediction $\hat{y}$ with respect to the true labels $y$ is given by:
</p>
$$
\begin{aligned}
L(\hat{y},y) =  -y.log\hat{y} - (1-y) . log(1-\hat{y})
\end{aligned}
$$
<p style="font-family: Arial; font-size:1.2em;color:black;">
For all samples:
</p>
$$
\begin{aligned}
J(\hat{y}, y) =  -\frac{1}{m}\sum_{i \in m}y_i.log\hat{y_i} - (1-y_i) . log(1-\hat{y_i})
\end{aligned}
$$
<p style="font-family: Arial; font-size:1.2em;color:black;">
    We can use gradient descent to find its minimum. for purpose of this demonstration lets use it in its simplest form - <strong>batch gradient descent with fixed learning rate</strong>.
    </p>

<p style="font-family: Arial; font-size:1.2em;color:black;">
    We will be using $tanh$ function for layer 1 (hidden layer) as it is most common which fits in majority of cases and its derivative can simply be represented as 1 -$\tanh^2(z_1)$. Since our output is binary, it makes sense to use $\mathrm{Softmax} / \mathrm{Sigmoid}$.
</p>
<img src='../Presentations/images/S5/nn_S5_f2.jpg' style='width: 100'/>

## Forward Propagation
$
\begin{aligned}
z_1^{[1]} & = x . W_1^{[1]} + b_1^{[1]}\\
a_1^{[1]} & = \tanh(z_1^{[1]}) \\
\\
z_2^{[1]} & = x . W_2^{[1]} + b_2^{[1]} \\
a_2^{[1]} & = \tanh(z_2^{[1]}) \\
\\
z_3^{[1]} & = x . W_3^{[1]} + b_3^{[1]} \\
a_3^{[1]} & = \tanh(z_3^{[1]}) \\
\\
z_4^{[1]} & = x . W_4^{[1]} + b_4^{[1]} \\
a_4^{[1]} & = \tanh(z_4^{[1]}) \\
\end{aligned}
$
<hr>
<p style="font-family: Arial; font-size:1.2em;color:black;">
    If we convert above to matrix version, we can say.</p>
$
\begin{aligned}
Z^{[1]} & = X . W^{[1]} + b^{[1]} \\
a^{[1]} & = \tanh(z^{[1]}) \\
\end{aligned}
$

<p style="font-family: Arial; font-size:1.2em;color:black;">
    Similarly for second layer.
</p>

$
\begin{aligned}
z^{[2]} & = a^{[1]}. W^{[2]} + b^{[2]} \\
a^{[2]} & = \hat{y} = \mathrm{softmax}(z^{[2]})\\
\end{aligned}
$

<p style="font-family: Arial; font-size:1.2em;color:black;">
    Where:
</p>
$
\begin{aligned}
\mathrm{softmax}(z_i) & =  \frac{e^{z_i}}{\sum_{i=1}^n {e^{z_i}}}\\
\end{aligned}
$
<p> Above derivation covers all layers and one training row.</p>

In [ ]:
# Helper function to evaluate the total loss on the dataset

def calculate_loss(model, X, y):
    
    # extract weights and losses from the model
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    # Forward propagation to calculate our predictions
    z1 = X.dot(W1) + b1
    
    # Tanh activation
    a1 = np.tanh(z1)
    
    z2 = a1.dot(W2) + b2
    
    # softmax activation
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    
    # Calculating the loss
    # Cross entropy = ground truth x log (predicted)
    # probability of y being correct is 1. hence it will be a vector of [1,1,...,1,1]
    
    correct_logprobs = -np.log(probs[range(num_examples), y]) 
    data_loss = np.sum(correct_logprobs)

    
    return 1./num_examples * data_loss

## Predict Function
<p style="font-family: Arial; font-size:1.2em;color:black;">
For predictions, we will simply be using the forward propagation. No need to iterate or calculate the back propagation for supervised learning.
</p>

In [ ]:
# Helper function to predict an output (0 or 1)

def predict(model, x):
    '''
     Args:
         model
         x: input features
    '''
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    # Forward propagation
    z1 = x.dot(W1) + b1
    
    a1 = np.tanh(z1)
    
    z2 = a1.dot(W2) + b2
    
    # use softmax
    exp_scores = np.exp(z2)
    
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    
    return np.argmax(probs, axis=1) # pick with one with highest probabilities

<img src='../Presentations/images/S5/nn_S5_f3.jpg' style='width: 800px;'/>

<p style="font-family: Arial; font-size:1.2em;color:black;">
As an input, gradient descent needs the gradients (vector of derivatives) of the loss function with respect to our parameters: $\frac{\partial{L}}{\partial{W_1}}(= \partial{W^{[1]}})$, $\frac{\partial{L}}{\partial{b_1}}(= \partial{b^{[1]}})$, $\frac{\partial{L}}{\partial{W_2}}(= \partial{W^{[2]}})$, $\frac{\partial{L}}{\partial{b_2}}(= \partial{b^{[2]}})$. To calculate these gradients we use the <strong>backpropagation algorithm,</strong>. Following calculations are Matrix form of single-row-equations shown above.
    </p>

## Backpropagation

$
\begin{aligned}
\partial{Z^{[2]}}  & = A^{[2]} - Y \\
\partial{W^{[2]}}  & = \frac{1}{m} A^{[1]T}\circ \partial{Z^{[2]}} \\
\partial{b^{[2]}}  & = \frac{1}{m} \mathrm{np.sum}(\partial{Z^{[2]}}, axis = 1, keepdims = True) \\
\\
\partial{Z^{[1]}}  & = \partial{Z^{[2]}}\circ  W^{[2]T} * ( 1-A^{[1]}**2)\\
\partial{W^{[1]}}  & = \frac{1}{m} X^{T}\circ \partial{Z^{[1]}} \\
\partial{b^{[1]}}  & = \frac{1}{m} \mathrm{np.sum}(\partial{Z^{[1]}}, axis = 1, keepdims = True) \\
\\
\end{aligned}
$

In [ ]:
# prepare the Model

def build_model(nn_hdim, X, y, EPOCHS=20000,  print_loss=False):
    
    '''
        nn_hdim : Number of nodes in the hidden layer
        X : Features to train on
        y : Targets to train on
        EPOCHS : Number of passes through the training data for gradient descent
        print_loss : If True, print the loss every nnn iterations
    '''
    # set Random Seed
    np.random.seed(RANDOM_STATE)
    
    # Initialize the parameters to random values. We need to learn these.
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))

    # This is what we return at the end
    model = {}
    curr_loss = 0
    loss = []
    epoch = []
    
    # Gradient descent. For each batch...
    for i in range(0, EPOCHS):
        
        ##########################
        #   Forward propagation  #
        ##########################
        
        # Layer 1
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)  # tanh activation function for layer 1
        
        # Layer 2
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2) # softmax for final layer as it is binary classification
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        #######################
        #   Back propagation  #
        #######################
        # Layer 2
        dz2 = probs
        dz2[range(num_examples), y] -= 1 # dL/db = dL/dz = (a-y). 
        #                                As Y is single dimension subtract one from its class
        
        dW2 = (a1.T).dot(dz2)/num_examples
        assert(W2.shape == dW2.shape), 'Shape of W2 {} and dW2 {} do not match'.format(W2.shape, dW2.shape)
        
        db2 = np.sum(dz2, axis=0, keepdims=True) / num_examples # db2 is vertical sum
        assert(b2.shape == db2.shape), 'Shape of b2 {} and db2 {} do not match'.format(b2.shape, db2.shape)
            
        dz1 = dz2.dot(W2.T) * (1 - np.power(a1, 2))  #derivative of tanh is (1−tanh(x)**2)
        assert(z1.shape == dz1.shape), 'Shape of z1 {} and dz1 {} do not match'.format(W2.shape, dW2.shape)
        
        dW1 = np.dot(X.T, dz1)/num_examples
        assert(W1.shape == dW1.shape), 'Shape of W1 {} and dW1 {} do not match'.format(W1.shape, dW1.shape)
        db1 = np.sum(dz1, axis=0)/num_examples # Note changing dim of db1
        
        
        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        if i % 100:
            curr_loss = calculate_loss(model, X, y)
            loss.append(curr_loss)
            epoch.append(i)
        
        # Print the loss.
        if print_loss and i % 1000 == 0:
            print("Loss after iteration %i: %f" %(i, curr_loss))
            
    curr_loss = calculate_loss(model, X, y)
    loss.append(curr_loss)
    epoch.append(i)
    print("Loss after iteration %i: %f" %(i, curr_loss))
    
    loss_hist['epoch'] = epoch
    loss_hist['loss'] = loss
    
    return model, probs

In [ ]:
num_examples = len(X_train) # training set size
nn_input_dim = 2 # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality

# lists to facilitate plotting 
loss_hist = {}

# Gradient descent parameters
# Try following values of epsilon to see its effect on the graph
#[ 0.0001, 0.001, 0.1, 1]
epsilon = 0.1 # learning rate for gradient descent Try 
reg_lambda = 0.0 # regularization strength

In [ ]:
# Build a model with a 4-dimensional hidden layer
model, probs = build_model(4, X_train, y_train, EPOCHS = EPOCHS, print_loss=True)

In [ ]:
loss_df = pd.DataFrame(loss_hist)

fn_plot_decision_boundary(lambda x: predict(model, x), X_train, y_train) # plot decision boundary for this plot

plt.title("Decision Boundary");

In [ ]:
fig, axes = plt.subplots(1,2 , figsize = (15,6))

l_range = 100

ax = axes[0]

loss_df.plot(x = 'epoch', y = 'loss', ax = ax)
loss = loss_df['loss'].values

# little beautification
txtstr = "Errors: \n  Start : {:7.4f}\n   End : {:7.4f}".format(loss[0],loss[-1]) #text to plot
# properties  matplotlib.patch.Patch 
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords

ax.text(0.6, 0.95, txtstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

ax.set_xlabel("Epochs")
ax.set_ylabel("Error")
ax.set_title('Overall')
ax.grid();

ax = axes[1]

loss_df[-l_range:].plot(x = 'epoch', y = 'loss', ax = ax)

# little beautification
txtstr = "Errors: \n  Start : {:7.4f}\n   End : {:7.4f}".format(loss[-l_range],loss[-1]) #text to plot
# properties  matplotlib.patch.Patch 
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords

ax.text(0.6, 0.95, txtstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

ax.set_xlabel("Epochs")
ax.set_ylabel("Error")
ax.set_title('Last {} records'.format(l_range))
ax.grid();
plt.tight_layout()

In [ ]:
def fn_make_predicitions(pred_func, X):
    y_pred = pred_func(X)
    return y_pred

In [ ]:
y_pred = fn_make_predicitions(lambda x: predict(model, x), X_train)
print('Accuracy score on Train Data :', accuracy_score(y_train, y_pred))

In [ ]:
y_pred = fn_make_predicitions(lambda x: predict(model, x), X_test)

print('Accuracy score on Test Data :', accuracy_score(y_test, y_pred))

In [ ]:
'''plt.figure(figsize=(10, 15)) # Set plot size

hidden_layer_dimensions = [1, 2, 3, 4, 5, 20] # number of layers to iterate

for i, nn_hdim in enumerate(hidden_layer_dimensions):

    plt.subplot(3, 2, i+1)
    
    plt.title('Hidden Layer size %d' % nn_hdim)
    
    model, probs = build_model(nn_hdim, X_train, y_train ,EPOCHS = EPOCHS,) # calling build model
    
    fn_plot_decision_boundary(lambda x: predict(model, x), X_train, y_train )
        

plt.tight_layout()
plt.show()'''

## Over Fitting

In [ ]:
plt.figure() # Set plot size

nn_hdim = 50 # number of layers to iterate

plt.subplot()

plt.title('Hidden Layer size %d' % nn_hdim)

model, probs = build_model(nn_hdim, X_train, y_train , EPOCHS = EPOCHS, print_loss = True) # calling build model

fn_plot_decision_boundary(lambda x: predict(model, x), X_train, y_train )


plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(4, activation='tanh'),
  tf.keras.layers.Dense(2)
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Gradient descent (with momentum) optimizer.

>tf.keras.optimizers.SGD(
>   learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD', **kwargs
>)


In [ ]:
model.compile(optimizer='SGD',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train , epochs = 5000, verbose = 0)

In [ ]:
model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
y_pred = probability_model(X_test).numpy().argmax(axis = 1)
print('Accuracy score on Test Data :', accuracy_score(y_test, y_pred))